# 2 | Partie Modélisation

### 1. Importation des librairies

In [14]:
import pandas as pd
import numpy as np

import missingno as msno

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from scipy.stats import zscore
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, StandardScaler, OrdinalEncoder, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE

import os
os.environ["PYTHONWARNINGS"] = "ignore"

### 2. Importation des données

In [15]:
df = pd.read_csv("dataset_assurance_cleaned.csv", )
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


In [16]:
### Encodage de nos variables qualitatives pour le schéma de corrélations


## Encodage N°1 avec BMI continu
df_encoded_1 = df.copy()

# Encodage de la variable 'smoker'
encodeur_smoker = LabelBinarizer()
encodeur_smoker.fit(df_encoded_1['smoker'])
df_encoded_1['smoker'] = encodeur_smoker.transform(df_encoded_1['smoker'])

# Encodage de la variable 'sex'
encodeur_sex = LabelBinarizer()
encodeur_sex.fit(df_encoded_1['sex'])
df_encoded_1['sex'] = encodeur_sex.transform(df_encoded_1['sex'])

# Encodage de la variable 'region'
df_encoded_1 = pd.get_dummies(df_encoded_1.copy(), columns=['region'], prefix=['region'], dtype=int)

# Renommage des colonnes
df_encoded_1.rename(columns={'sex': 'sex_male', 'smoker': 'smoker_yes'}, inplace=True)


In [17]:
df_encoded_1

,age,sex_male,bmi,children,smoker_yes,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1332,50,1,30.970,3,0,10600.54830,0,1,0,0
1333,18,0,31.920,0,0,2205.98080,1,0,0,0
1334,18,0,36.850,0,0,1629.83350,0,0,1,0
1335,21,0,25.800,0,0,2007.94500,0,0,0,1


In [18]:
## encodage N°2 avec BMI en catégories

# Création des catégories de BMI
def bmi_category(bmi):
    if bmi < 18:
        return "Sous-poids"
    elif bmi < 24.5:
        return "Normal"
    elif bmi < 30:
        return "Surpoids"
    elif bmi < 35:
        return "Obésité I"
    elif bmi < 40:
        return "Obésité II"
    else:
        return "Obésité III"


df_encoded_2 = df_encoded_1.copy()
df_encoded_2["bmi_cat"] = df_encoded_2["bmi"].apply(bmi_category)

# OrdinalEncoder pour encoder les catégories de manière ordonnée
ordinal_encoder = OrdinalEncoder(categories=[["Sous-poids", "Normal", "Surpoids", "Obésité I", "Obésité II", "Obésité III"]], dtype=int)

# Transformation des catégories
df_encoded_2["bmi_cat_numeric"] = ordinal_encoder.fit_transform(df_encoded_2[["bmi_cat"]])

# Suppression des autres colonnes BMI
df_encoded_2.drop(columns=['bmi', 'bmi_cat'], inplace=True)

In [19]:
df_encoded_2

,age,sex_male,children,smoker_yes,charges,region_northeast,region_northwest,region_southeast,region_southwest,bmi_cat_numeric
0,19,0,0,1,16884.92400,0,0,0,1,2
1,18,1,1,0,1725.55230,0,0,1,0,3
2,28,1,3,0,4449.46200,0,0,1,0,3
3,33,1,0,0,21984.47061,0,1,0,0,1
4,32,1,0,0,3866.85520,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...
1332,50,1,3,0,10600.54830,0,1,0,0,3
1333,18,0,0,0,2205.98080,1,0,0,0,3
1334,18,0,0,0,1629.83350,0,0,1,0,4
1335,21,0,0,0,2007.94500,0,0,0,1,2


In [20]:
## Encodage N°3 avec avec une catégorie (Obèse/pas obèse)

# Création des catégories de BMI
def bmi_category(bmi):
    if bmi < 30:
        return 0
    else:
        return 1
    
    
df_encoded_3 = df_encoded_1.copy()
df_encoded_3["obese_yes"] = df_encoded_3["bmi"].apply(bmi_category)
df_encoded_3.drop(columns=['bmi'], inplace=True)

In [21]:
df_encoded_3

,age,sex_male,children,smoker_yes,charges,region_northeast,region_northwest,region_southeast,region_southwest,obese_yes
0,19,0,0,1,16884.92400,0,0,0,1,0
1,18,1,1,0,1725.55230,0,0,1,0,1
2,28,1,3,0,4449.46200,0,0,1,0,1
3,33,1,0,0,21984.47061,0,1,0,0,0
4,32,1,0,0,3866.85520,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1332,50,1,3,0,10600.54830,0,1,0,0,1
1333,18,0,0,0,2205.98080,1,0,0,0,1
1334,18,0,0,0,1629.83350,0,0,1,0,1
1335,21,0,0,0,2007.94500,0,0,0,1,0


In [22]:
def run_model_comparison_with_polynomial(data, target_column='charges', scalers=None):
    """
    Compare plusieurs modèles de régression (Linéaire, Lasso, Ridge, ElasticNet) avec uniquement des caractéristiques polynomiales
    et recherche les meilleurs paramètres pour chaque modèle.

    Args:
        data (pd.DataFrame): Le DataFrame contenant les données.
        target_column (str): Le nom de la colonne cible (par défaut 'charges').
        scalers (list): Liste des scalers à tester (par défaut [StandardScaler, MinMaxScaler, RobustScaler]).

    Returns:
        pd.DataFrame: Résultats sous forme de DataFrame.
    """
    if scalers is None:
        scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]

    # Séparation des caractéristiques (X) et de la cible (y)
    X = data.drop(columns=[target_column])
    y = data[target_column]

    # Division en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X["smoker_yes"])

    # Dictionnaire pour stocker les résultats
    results = []

    # Liste des modèles à tester
    models = {
        'Linear Regression': LinearRegression(),
        'Lasso': Lasso(),
        'Ridge': Ridge(),
        'ElasticNet': ElasticNet()
    }

    # Grilles de recherche pour les hyperparamètres
    param_grids = {
        "Lasso": {
            "lasso__alpha": [1, 10, 100, 1000],
            "poly__degree": [2, 3, 4],  # Tester différents degrés de polynômes
            "lasso__max_iter": [1000]
        },
        "Ridge": {
            "ridge__alpha": [1, 10, 100, 1000],
            "poly__degree": [1, 2, 3, 4]
        },
        "ElasticNet": {
            "elasticnet__alpha": [0.1, 0.5, 1.0, 2.0, 5.0, 10.0],  # Valeurs possibles de alpha (la régularisation)
            "elasticnet__l1_ratio": [0.1, 0.25, 0.5, 0.75, 1.0],  # Ratio entre Lasso (L1) et Ridge (L2)
            "poly__degree": [1, 2, 3],  # Tester différents degrés de polynômes
            "elasticnet__max_iter": [1000]
        }
    }

    # Comparaison des résultats pour chaque scaler
    for scaler in scalers:
        scaler_name = scaler.__class__.__name__
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        for model_name, model in models.items():
            # Construction du pipeline avec PolynomialFeatures et le scaler
            pipeline = Pipeline([
                ('poly', PolynomialFeatures(include_bias=False)),  # Génération des polynômes
                ('scaler', scaler),                                # Standardisation
                (model_name.lower().replace(" ", "_"), model)      # Modèle (Lasso, Ridge, ElasticNet)
            ])

            # Si le modèle a des hyperparamètres à optimiser
            if model_name in param_grids:
                grid_search = GridSearchCV(
                    pipeline,
                    param_grids[model_name],
                    cv=5,
                    scoring='r2',
                    n_jobs=-1,
                    verbose=0
                )
                grid_search.fit(X_train_scaled, y_train)
                best_model = grid_search.best_estimator_
                best_params = grid_search.best_params_
                best_score = grid_search.best_score_
            else:
                # Pas de recherche d'hyperparamètres pour la régression linéaire simple
                pipeline.fit(X_train_scaled, y_train)
                best_model = pipeline
                best_params = None
                best_score = r2_score(y_train, pipeline.predict(X_train_scaled))

            # Calcul du score sur l'ensemble de test
            test_r2 = r2_score(y_test, best_model.predict(X_test_scaled))

            # Ajout des résultats dans le tableau
            results.append({
                'Scaler': scaler_name,  # Ajout du nom du scaler
                'Model': model_name,
                'Best Score (R2, Train)': best_score,
                'Test R2': test_r2,
                'Best Params': best_params
            })

    # Convertir les résultats en DataFrame
    return pd.DataFrame(results).sort_values('Test R2', ascending=False)


In [23]:
results_models = run_model_comparison_with_polynomial(df_encoded_3)

In [24]:
results_models

,Scaler,Model,"Best Score (R2, Train)",Test R2,Best Params
1,StandardScaler,Lasso,0.849064,0.920104,"{'lasso__alpha': 100, 'lasso__max_iter': 1000,..."
11,RobustScaler,ElasticNet,0.847516,0.918658,"{'elasticnet__alpha': 10.0, 'elasticnet__l1_ra..."
9,RobustScaler,Lasso,0.847516,0.918658,"{'lasso__alpha': 10, 'lasso__max_iter': 1000, ..."
3,StandardScaler,ElasticNet,0.846630,0.918551,"{'elasticnet__alpha': 10.0, 'elasticnet__l1_ra..."
6,MinMaxScaler,Ridge,0.846596,0.918426,"{'poly__degree': 2, 'ridge__alpha': 1}"
5,MinMaxScaler,Lasso,0.848687,0.918258,"{'lasso__alpha': 10, 'lasso__max_iter': 1000, ..."
7,MinMaxScaler,ElasticNet,0.848687,0.918258,"{'elasticnet__alpha': 10.0, 'elasticnet__l1_ra..."
10,RobustScaler,Ridge,0.846097,0.918069,"{'poly__degree': 2, 'ridge__alpha': 1}"
0,StandardScaler,Linear Regression,0.857609,0.918057,None
4,MinMaxScaler,Linear Regression,0.857609,0.918057,None


In [25]:
results_models.to_csv("general_results_models.csv", index=False)

In [26]:
results_models.pivot_table(index='Model', columns='Scaler', values='Test R2').sort_values(results_models['Test R2'],ascending=False)

KeyError: 1     0.920104
11    0.918658
9     0.918658
3     0.918551
6     0.918426
5     0.918258
7     0.918258
10    0.918069
0     0.918057
4     0.918057
8     0.918057
2     0.917816
Name: Test R2, dtype: float64

### 3. Les modèles

#### 3. 1. LinearRegression()

In [112]:
# Séparation des variables explicatives et de la variable cible
X = df_encoded_1.drop(columns='charges')
y = df_encoded_1['charges']

In [113]:
# Initialisation du modèle de régression linéaire
modelLR = LinearRegression()

In [114]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X["smoker_yes"])

In [ ]:
# Entraînement du modèle de régression linéaire
modelLR.fit(X_train, y_train)

In [116]:
# Prédiction sur l'ensemble de test
y_pred = modelLR.predict(X_test)

In [ ]:
# Evaluation de la performance du modèle

### MSE  - Erreur quadrique moyenne : mesure l'erreur entre les valeurs réelles et les valeurs prédies
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


### R²   - Coefficient de détermination : mesure la proportion de la variance des données qui est expliquée par le modèle
r2 = modelLR.score(X_test, y_test)
print(f"R²: {r2}")


### MAE  - Erreur Absolue Moyenne : la miyenne des erreurs absolues entre les prédictions et les valeurs réelles
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

* R² indique que le modèle est relativement performant, expliquant presque 80 % de la variance des données.
* Le MSE suggère que les erreurs sont relativement grandes en moyenne, mais cela peut être acceptable selon la portée de le problème.
* Le MAE montre que, en moyenne, le modèle fait une erreur d'environ 4,043 unités par prédiction.

In [ ]:
coefficients_LR = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': modelLR.coef_
})

coefficients_LR.sort_values('Variable')

* Variables influentes : Le fait d'être fumeur (smoker_encoded), l'âge (age), et le BMI (bmi) ont des effets 'positifs' significatifs sur la variable cible.
* Variables modérées : Le nombre d'enfants (children) a un effet 'positif', mais plus faible en comparaison avec d'autres variables comme le BMI ou le statut de fumeur.
* Variables moins influentes : Le sexe (sex_encoded) et la région (region_encoded) ont des effets 'négatifs' sur la variable cible.

#### Fonction pour le ML de la Régression Linéaire

In [119]:
def train_linear_regression_model(data, target_column):
    """
    Implémente un modèle de régression linéaire avec Scikit-learn.
    
    Args:
        data (pd.DataFrame): Le DataFrame contenant les données.
        target_column (str): Le nom de la colonne cible.
        test_size (float): La proportion des données à utiliser pour le test (par défaut : 0.2).
        random_state (int): La graine pour la reproductibilité (par défaut : 42).
    
    Returns:
        dict: Un dictionnaire contenant les métriques d'évaluation et le modèle entraîné.
    """
    # Séparation des caractéristiques (X) et de la cible (y)
    X = data.drop(columns=[target_column])
    y = data[target_column]
    
    # Division en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X["smoker_yes"])
    
    # Initialisation du modèle
    model = LinearRegression()
    
    # Entraînement du modèle
    model.fit(X_train, y_train)
    
    # Prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)
    
    # Calcul des métriques
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Résumé des résultats
    results = {
        "model": model,
        "mean_squared_error": mse,
        "r2_score": r2,
        "coefficients": model.coef_,
        "intercept": model.intercept_,
    }
    
    return results

In [ ]:
train_linear_regression_model(df_encoded_2, 'charges')

In [ ]:
train_linear_regression_model(df_encoded_3, 'charges')

### 3. 2. Model Lasso

#### 3.2.1 Méthode Standard

In [ ]:
# Initialisation et entrainement du Lasso
lasso_model = Lasso(alpha=1.0)
lasso_model.fit(X_train, y_train)

In [123]:
# Prédiction sur l'ensemble du test
y_pred = lasso_model.predict(X_test)

In [ ]:
# Evalusation du modèle
mse = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R²: {r2}")

In [ ]:
# Affichage des coefficients
coefficients_lasso = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': lasso_model.coef_
}).sort_values(by='Variable')

coefficients_lasso

In [126]:
def train_lasso_regression_model(data, target_column='charges', alpha=1.0):
    """
    Implémente un modèle de régression Lasso avec Scikit-learn.
    
    Args:
        data (pd.DataFrame): Le DataFrame contenant les données.
        target_column (str): Le nom de la colonne cible.
        alpha (float): Le paramètre de régularisation (par défaut : 1.0).
        test_size (float): La proportion des données à utiliser pour le test (par défaut : 0.2).
        random_state (int): La graine pour la reproductibilité (par défaut : 42).
    
    Returns:
        dict: Un dictionnaire contenant les métriques d'évaluation et le modèle entraîné.
    """
    # Séparation des caractéristiques (X) et de la cible (y)
    X = data.drop(columns=[target_column])
    y = data[target_column]
    
    # Division en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X["smoker_yes"])
    
    # Initialisation du modèle Lasso
    model = Lasso(alpha=alpha)
    
    # Entraînement du modèle
    model.fit(X_train, y_train)
    
    # Prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)
    
    # Calcul des métriques
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Résumé des résultats
    results = {
        "model": model,
        "mean_squared_error": mse,
        "r2_score": r2,
        "coefficients": model.coef_,
        "intercept": model.intercept_,
    }
    
    return results


In [ ]:
train_lasso_regression_model(df_encoded_2)

In [ ]:
train_lasso_regression_model(df_encoded_3)

#### 3.2.2 StandardScaler

Standardisation : Centrage et réduction pour avoir une moyenne de 0 et un écart-type de 1.
Pour la régression Lasso, la standardisation est souvent recommandée car elle maintient les distributions tout en ajustant l'échelle.

1. **RobustScaler** :
* Sensibilité aux outliers : Résistant (utilise la médiane et l'IQR, écart interquartile).
* Échelle des données : Basée sur l'IQR (interquartile range).
* Quand l'utiliser ? : Adapté aux données contenant des outliers ou des distributions asymétriques.

2. **StandardScaler** :
* Sensibilité aux outliers : Sensible (utilise la moyenne et l’écart-type).
* Échelle des données : Centre les données à une moyenne de 0 et réduit leur écart-type à 1.
* Quand l'utiliser ? : Adapté aux données sans outliers, ou lorsque les données suivent une distribution normale.

3. **MinMaxScaler** :
* Sensibilité aux outliers : Très sensible (utilise les valeurs min et max).
* Échelle des données : Transforme les données dans une plage définie, généralement [0, 1].
* Quand l'utiliser ? : Utile lorsque les données doivent être dans une plage spécifique (par exemple pour les réseaux neuronaux ou les algorithmes nécessitant des valeurs normalisées).

In [129]:
# Standardisation

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
lasso_model = Lasso(alpha=1.0)
lasso_model.fit(X_train_scaled, y_train)

In [131]:
y_pred = lasso_model.predict(X_test_scaled)

In [ ]:
# Evalusation du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R²: {r2}")

In [ ]:
# Affichage des coefficients
coefficients_lasso = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': lasso_model.coef_
}).sort_values(by='Variable')

coefficients_lasso

#### Fonction pour le modèle de régression lasso

In [134]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

def train_lasso_regression_model_with_scaler(data, target_column, alphas=None, test_size=0.2, random_state=42, scaler=None):
    """
    Implémente un modèle de régression Lasso avec choix du scaler et recherche d'hyperparamètres via GridSearchCV.
    
    Args:
        data (pd.DataFrame): Le DataFrame contenant les données.
        target_column (str): Le nom de la colonne cible.
        alphas (list): Liste des valeurs d'alpha à tester (par défaut : [0.01, 0.1, 1, 10, 100]).
        test_size (float): La proportion des données à utiliser pour le test (par défaut : 0.2).
        random_state (int): La graine pour la reproductibilité (par défaut : 42).
        scaler (object): Un scaler Scikit-learn (par ex. StandardScaler, MinMaxScaler, RobustScaler). Si None, aucune mise à l'échelle n'est effectuée.
    
    Returns:
        dict: Un dictionnaire contenant les métriques d'évaluation, le modèle entraîné, et le meilleur alpha.
    """
    if alphas is None:
        alphas = [0.01, 0.1, 1, 10, 100]
    
    # Séparation des caractéristiques (X) et de la cible (y)
    X = data.drop(columns=[target_column])
    y = data[target_column]
    
    # Division en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Construction du pipeline
    steps = []
    if scaler is not None:
        steps.append(("scaler", scaler))
    steps.append(("lasso", Lasso(random_state=random_state)))
    pipeline = Pipeline(steps)
    
    # Recherche du meilleur hyperparamètre alpha avec validation croisée
    param_grid = {'lasso__alpha': alphas}
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')
    grid_search.fit(X_train, y_train)
    
    # Meilleur modèle et alpha
    best_model = grid_search.best_estimator_
    best_alpha = grid_search.best_params_['lasso__alpha']
    
    # Prédictions sur l'ensemble de test
    y_pred = best_model.predict(X_test)
    
    # Calcul des métriques
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Résumé des résultats
    results = {
        "model": best_model,
        "mean_squared_error": mse,
        "r2_score": r2,
        "coefficients": best_model.named_steps['lasso'].coef_,
        "intercept": best_model.named_steps['lasso'].intercept_,
        "best_alpha": best_alpha
    }
    
    return results


#### 3.2.3 Sélection du meilleur alpha

Utilisation de la validation croisée avec une recherche de grille (GridSearchCV) pour tester différentes valeurs d'alpha et choisir celle qui minimise l'erreur de validation.

In [135]:
# Définition de la plage des valeurs d'alpha à tester
param_grid = {
                'alpha' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                'max_iter': [5000, 10000, 50000],
                'tol': [1e-3, 1e-2]
}


In [136]:
# Effectuer une recherche de grille avec validation croisée 

lasso_gs = GridSearchCV(lasso_model, param_grid, cv=5, scoring='r2', n_jobs=-1)

In [ ]:
lasso_gs.fit(X_train_scaled, y_train)

In [ ]:
# Afficher le meilleur alpha et ses performances
print(f"Meilleur alpha trouvé : {lasso_gs.best_params_['alpha']}, {lasso_gs.best_estimator_}")

In [ ]:
# Meilleur modèle
best_lasso_model = lasso_gs.best_estimator_
best_lasso_model 

In [ ]:
# Test sur le nouveau modèle ajusté
best_lasso_model.fit(X_train_scaled, y_train)
y_pred = best_lasso_model.predict(X_test_scaled)

# Evalusation du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R²: {r2}")

In [ ]:
# Affichage des coefficients
coefficients_lasso = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': lasso_model.coef_
}).sort_values(by='Variable')

coefficients_lasso

#### Fonction Modèle ML Lasso avec recherche d'hyperparamètres

In [142]:

def train_lasso_regression_model_with_cv(data, target_column='charges', alphas=None):
    """
    Implémente un modèle de régression Lasso avec sélection du meilleur alpha via GridSearchCV.
    
    Args:
        data (pd.DataFrame): Le DataFrame contenant les données.
        target_column (str): Le nom de la colonne cible.
        alphas (list): Liste des valeurs d'alpha à tester (par défaut : [0.01, 0.1, 1, 10, 100]).
        test_size (float): La proportion des données à utiliser pour le test (par défaut : 0.2).
        random_state (int): La graine pour la reproductibilité (par défaut : 42).
    
    Returns:
        dict: Un dictionnaire contenant les métriques d'évaluation, le modèle entraîné, et le meilleur alpha.
    """
    if alphas is None:
        alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    
    # Séparation des caractéristiques (X) et de la cible (y)
    X = data.drop(columns=[target_column])
    y = data[target_column]
    
    # Division en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X["smoker_yes"])
    
    # Initialisation du modèle Lasso
    model = Lasso()
    
    # Recherche du meilleur hyperparamètre alpha avec validation croisée
    param_grid = {'alpha': alphas}
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='r2')
    grid_search.fit(X_train, y_train)
    
    # Meilleur modèle et alpha
    best_model = grid_search.best_estimator_
    best_alpha = grid_search.best_params_['alpha']
    
    # Prédictions sur l'ensemble de test
    y_pred = best_model.predict(X_test)
    
    # Calcul des métriques
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Résumé des résultats
    results = {
        "model": best_model,
        "mean_squared_error": mse,
        "r2_score": r2,
        "coefficients": best_model.coef_,
        "intercept": best_model.intercept_,
        "best_alpha": best_alpha
    }
    
    return results


In [ ]:
train_lasso_regression_model_with_cv(df_encoded_2)

In [ ]:
train_lasso_regression_model_with_cv(df_encoded_3)

## 2. Transformation polynomiale

Pour améliorer les performances du modèle de régression (Lasso ou autre), nous pouvons appliquer une transformation polynomiale sur vos variables explicatives. Cela permet de capturer des relations non linéaires entre les variables indépendantes et la variable cible.

#### 2.1.1 Façon Standard

In [145]:
# Pipeline avec PolynomialFeatures
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Génération des polynômes
    ('scaler', StandardScaler()),                                # Standardisation
    ('lasso', Lasso(alpha=10, max_iter=5000))                    # Modèle Lasso
])

In [146]:
# Entraînement du modèle
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)

In [ ]:
# Évaluation du modèle
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R²: {r2}")

#### 2.1.2 Optimisation des hyperparamètres avec GridSearchCV

In [148]:
# Grille des paramètres

param_grid = {
    'poly__degree': [1, 2, 3],                              # Tester différents degrés de polynômes
    'lasso__alpha': range(1, 100),   # Tester différentes valeurs de régularisation
    'lasso__max_iter': [1000, 5000, 10000]                  
}

In [ ]:
# GridSearchCV
model_poly = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
model_poly.fit(X_train, y_train)

In [ ]:
# Meilleur modèle
best_model_poly = model_poly.best_estimator_
best_model_poly

In [ ]:
# Prédictions avec le meilleur modèle
y_pred = best_model_poly.predict(X_test)

# Évaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Parameters: {model_poly.best_params_}")
print(f"Mean Squared Error: {mse}")
print(f"R²: {r2}")



'''Premier essai --> alpha=10'''

In [ ]:
# Initialisation
# Pipeline avec PolynomialFeatures
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Génération des polynômes
    ('scaler', StandardScaler()),                                # Standardisation
    ('lasso', Lasso(alpha=21, max_iter=1000))                    # Modèle Lasso
])

# Entraînement du modèle
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)

# Évaluation du modèle
mse_ridge = mean_squared_error(y_test, y_pred)
r2_ridge = r2_score(y_test, y_pred)

print(f"Mean Squared Error (Ridge): {mse_ridge}")
print(f"R² (Ridge): {r2_ridge}")

## 3. Modèle Ridge

In [ ]:
# Initialisation
ridge_model = Ridge(alpha=1.0, max_iter=5000)

# Entraînement du modèle sur les données standardisées
ridge_model.fit(X_train_scaled, y_train)

# Prédictions sur l'ensemble de test
y_pred_ridge = ridge_model.predict(X_test_scaled)

# Évaluation du modèle
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print(f"Mean Squared Error (Ridge): {mse_ridge}")
print(f"R² (Ridge): {r2_ridge}")

In [ ]:
coefficients_ridge = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': ridge_model.coef_
}).sort_values(by='Variable')

coefficients_ridge

In [ ]:
# Paramètres
param_grid_ridge = {
    'alpha': range(1, 100),
    'max_iter': [1000, 5000, 10000]
}

# Initialisation de GridSearchCV
ridge_grid_search = GridSearchCV(
    estimator=Ridge(),
    param_grid=param_grid_ridge,
    cv=5,
    n_jobs=-1
)

# Entraînement de la recherche de grille
ridge_grid_search.fit(X_train_scaled, y_train)

# Meilleur modèle Ridge
best_ridge_model = ridge_grid_search.best_estimator_

# Prédictions avec le meilleur modèle
y_pred_best_ridge = best_ridge_model.predict(X_test_scaled)

# Évaluation du meilleur modèle
mse_best_ridge = mean_squared_error(y_test, y_pred_best_ridge)
r2_best_ridge = r2_score(y_test, y_pred_best_ridge)

print(f"Best Alpha (Ridge): {ridge_grid_search.best_params_['alpha']}")
print(f"Mean Squared Error (Best Ridge): {mse_best_ridge}")
print(f"R² (Best Ridge): {r2_best_ridge}")


'''Premier essai --> alpha=10'''

In [ ]:
# Initialisation
ridge_model = Ridge(alpha=7, max_iter=5000)

# Entraînement du modèle sur les données standardisées
ridge_model.fit(X_train_scaled, y_train)

# Prédictions sur l'ensemble de test
y_pred_ridge = ridge_model.predict(X_test_scaled)

# Évaluation du modèle
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print(f"Mean Squared Error (Ridge): {mse_ridge}")
print(f"R² (Ridge): {r2_ridge}")

## 4. ElasticNet

* Best Score (R2)	
    - Pendant la validation croisée	
    - Données d'entraînement (via CV)	
    - Indique la performance maximale du modèle sur les sous-ensembles de données d'entraînement, avant de finaliser l'entraînement.
* Test R2	
    - Après l'entraînement complet	
    - Données de test (non vues)	
    - Indique la capacité du modèle à généraliser et à prédire sur des données nouvelles qu'il n'a jamais vues.

In [ ]:
run_model_comparison_with_polynomial(df_encoded_1)

In [ ]:
run_model_comparison_with_polynomial(df_encoded_2)

In [ ]:
run_model_comparison_with_polynomial(df_encoded_3)